In [1]:

!pip install cdflib numpy pandas matplotlib plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 1.9 MB/s eta 0:00:00


In [2]:
!pip install spacepy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 27.5 MB/s eta 0:00:00


**Data collecting**

In [3]:
!ls

sample_data


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os

folder_path = '/content/drive/MyDrive/cme'  # root folder in Colab
files = [f for f in os.listdir(folder_path) if f.endswith('.cdf')]
files = sorted(files)  # optional: ensures chronological order
print(files)

['AL1_ASW91_L1_AUX_20251017_UNP_9999_999999_V01.cdf', 'AL1_ASW91_L1_TH1_20251017_UNP_9999_999999_V01.cdf', 'AL1_ASW91_L1_TH2_20251017_UNP_9999_999999_V01.cdf', 'AL1_ASW91_L2_BLK_20251015_UNP_9999_999999_V02.cdf', 'AL1_ASW91_L2_BLK_20251017_UNP_9999_999999_V02.cdf', 'AL1_ASW91_L2_TH1_20251015_UNP_9999_999999_V02.cdf', 'AL1_ASW91_L2_TH1_20251017_UNP_9999_999999_V02.cdf', 'AL1_ASW91_L2_TH2_20251015_UNP_9999_999999_V02.cdf', 'AL1_ASW91_L2_TH2_20251017_UNP_9999_999999_V02.cdf']


**Data Processing**

In [6]:
import cdflib

cdf_file = cdflib.CDF('/content/AL1_ASW91_L1_AUX_20251017_UNP_9999_999999_V01.cdf')  # replace with one of your files
print(cdf_file.cdf_info())  # lists all variable names


FileNotFoundError: /content/AL1_ASW91_L1_AUX_20251017_UNP_9999_999999_V01.cdf not found

In [8]:
import os
import cdflib
import numpy as np
import pandas as pd

folder_path = '/content/drive/MyDrive/cme'  # root folder
files = [f for f in os.listdir(folder_path) if f.endswith('.cdf')]
files = sorted(files)

# Pick one file (event-day or pre-event)
f = files[0]
cdf_file = cdflib.CDF(os.path.join(folder_path, f))

# List all variable names (zVariables and rVariables)
cdf_info = cdf_file.cdf_info()
z_vars = cdf_info.zVariables  # already a list of strings
r_vars = cdf_info.rVariables  # already a list of strings

print("ZVariables:", z_vars)
print("RVariables:", r_vars)

# Pick the first available numeric variable from zVariables
var_name = z_vars[0]  # you can change this
print("Using variable:", var_name)

# Load numeric data safely
data_array = np.array(cdf_file.varget(var_name))
data_array = data_array[data_array != -1.0e31]  # remove fill values
data_array = data_array.flatten()

# Create a simple time index
time_index = np.arange(len(data_array))

# Make a DataFrame
data = pd.DataFrame({
    'time': time_index,
    var_name: data_array
})

print(data.head())
print("Total rows:", len(data))




ZVariables: ['obs_time', 'epoch_for_cdf', 'fpga_ticks', 'frame_num', 'last_comm_fpga', 'trig_counts', 'coin_trig_counts', 'peak_det_counts', 'rej_counts', 'fpga_temp_mon', 'fpga_core_volt_mon', 'fpga_io_volt_mon', 'FEE_volt_mon', 'DC_temp_mon', 'HV_pack_mon', 'ESA1_HV_set', 'ESA2_HV_set', 'Focus_set', 'MCP1_HV_set', 'MCP2_HV_set', 'Focus', 'MCP1_HV', 'MCP1_curr_mon', 'ESA1_HV', 'HV_PCB_temp_mon', 'MCP2_HV_mon', 'MCP2_curr_mon', 'ESA2_HV_mon', 'HW_LLD', 'SW_LLD', 'SW_ULD', 'SW_CSL', 'SW_CSU', 'coinc_trig_count_total', 'count_UL_auto', 'count_LL_auto', 'pile_up_time', 'step_num', 'over_curr_limit_MCP1', 'over_curr_limit_MCP2', 'cal_hist_mode', 'auto_ctrl_ch_mode', 'THA2_single_energy_mode', 'THA1_single_energy_mode', 'FEE_cal_chn', 'FEE_cal_mode', 'MCP2_PHD_mode', 'MCP1_PHD_mode', 'auto_ctrl_mode', 'man_ctrl_mode', 'comm_mode', 'operating_mode', 'MCP1_spike_override', 'MCP2_spike_override', 'MCP1_spike_trip', 'MCP2_spike_trip', 'MCP1_start_comm', 'MCP2_start_comm', 'MCP1_status', 'MCP2_s

In [9]:
from sklearn.preprocessing import MinMaxScaler

In [13]:
# Example: combine multiple features
features = [
    'trig_counts',
    'coin_trig_counts',
    'peak_det_counts',
    'rej_counts',
    'fpga_temp_mon',
    'ESA1_HV_set',
    'ESA2_HV_set',
    'spacecraft_xvel'
]

data = []
for f in features:
    arr = np.array(cdf_file.varget(f))
    arr = arr[arr != -1.0e31].flatten()
    data.append(arr)

# Make sure all features are same length
min_len = min(len(a) for a in data)
data = [a[:min_len] for a in data]
feature_matrix = np.vstack(data).T  # shape: (timesteps, num_features)

# Scale all features
scaler = MinMaxScaler()
feature_scaled = scaler.fit_transform(feature_matrix)

# Create sequences
sequence_length = 50
X = []
for i in range(len(feature_scaled)-sequence_length):
    X.append(feature_scaled[i:i+sequence_length])
X = np.array(X)

**Model building**

In [14]:
!pip install --upgrade tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 6.3 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboard-2.19.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.19.0
    Uninstalling tensorflow-2.19.0:
      Successfully uninstalled tensorflow-2.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.20.0 which is incompatible.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.20.0 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.20.0 which is inco

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

print(tf.__version__)

2.20.0


In [18]:
import numpy as np
import pandas as pd

# Load variables
epoch = np.array(cdf_file.varget('epoch_for_cdf')).flatten()
trig = np.array(cdf_file.varget('trig_counts')).flatten()
coin = np.array(cdf_file.varget('coin_trig_counts')).flatten()
temp = np.array(cdf_file.varget('fpga_temp_mon')).flatten()

# Find shortest length
min_len = min(len(epoch), len(trig), len(coin), len(temp))

# Trim everything to the same length
epoch = epoch[:min_len]
trig = trig[:min_len]
coin = coin[:min_len]
temp = temp[:min_len]

# Build DataFrame
data = pd.DataFrame({
    'epoch_for_cdf_mod': epoch,
    'trig_counts': trig,
    'coin_trig_counts': coin,
    'fpga_temp_mon': temp
})

# Clean fill values
fill_val = -1.0e31
data.replace(fill_val, np.nan, inplace=True)
data.dropna(inplace=True)

print("✅ DataFrame ready! Shape:", data.shape)
print(data.head())


✅ DataFrame ready! Shape: (172786, 4)
   epoch_for_cdf_mod  trig_counts  coin_trig_counts  fpga_temp_mon
0       6.392788e+13            0                 0           1771
1       6.392788e+13            0                 0           1771
2       6.392788e+13            2                 2           1771
3       6.392788e+13            3                 3           1771
4       6.392788e+13           67                62           1771


In [19]:
trig_thresh = np.percentile(data['trig_counts'], 90)
coin_thresh = np.percentile(data['coin_trig_counts'], 90)
temp_thresh = np.percentile(data['fpga_temp_mon'], 85)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

# Build simple LSTM model
model = Sequential([
    LSTM(32, input_shape=(X.shape[1], X.shape[2])),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train quickly (few epochs for demo)
model.fit(X, y, epochs=3, batch_size=1024)



/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 32)             │         4,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,385 (17.13 KB)

 Trainable params: 4,385 (17.13 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
8437/8437 ━━━━━━━━━━━━━━━━━━━━ 863s 102ms/step - accuracy: 0.5455 - loss: 0.6653
Epoch 2/3
8437/8437 ━━━━━━━━━━━━━━━━━━━━ 821s 97ms/step - accuracy: 0.6124 - loss: 0.6079
Epoch 3/3
8437/8437 ━━━━━━━━━━━━━━━━━━━━ 815s 96ms/step - accuracy: 0.5837 - loss: 0.6347


In [ ]:
# Get prediction probabilities (values between 0 and 1)
pred_probs = model.predict(X, verbose=1)

# Use a stricter threshold for CME detection
pred_labels = (pred_probs > 0.9).astype(int)

print("First 20 predictions:", pred_labels[:20].flatten())

# Optional: check how many are CME vs non-CME
unique, counts = np.unique(pred_labels, return_counts=True)
print("Label distribution:", dict(zip(unique, counts)))

269977/269977 ━━━━━━━━━━━━━━━━━━━━ 1085s 4ms/step
First 20 predictions: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label distribution: {np.int64(0): np.int64(8639250)}


In [ ]:
sequence_length = 50  # same as during training
cme_indices = np.where(pred_labels.flatten() == 1)[0]

if len(cme_indices) > 0:
    cme_start_idx = cme_indices[0] + sequence_length - 1
    cme_end_idx = cme_indices[-1] + sequence_length - 1

    cme_end_idx = min(cme_end_idx, len(data) - 1)

    print(f"Predicted CME from index {cme_start_idx} to {cme_end_idx}")
    print(f"That covers about {cme_end_idx - cme_start_idx + 1} samples.")
else:
    print("No CME predicted in this data.")



Predicted CME from index 49 to 863929
That covers about 863881 samples.


In [ ]:
from datetime import datetime

sequence_length = 50  # same as when you created sequences
cme_indices = np.where(pred_labels.flatten() == 1)[0]

if len(cme_indices) > 0:
    cme_start_idx = cme_indices[0] + sequence_length - 1
    cme_end_idx = cme_indices[-1] + sequence_length - 1
    cme_end_idx = min(cme_end_idx, len(data) - 1)

    # Get timestamps from the data
    cme_start_time = data['obs_time'].iloc[cme_start_idx]
    cme_end_time = data['obs_time'].iloc[cme_end_idx]

    # Convert CDF timestamps to readable datetime
    start_time = datetime.utcfromtimestamp(cme_start_time)
    end_time = datetime.utcfromtimestamp(cme_end_time)

    # Duration
    duration_hours = (end_time - start_time).total_seconds() / 3600

    # Number of detected points
    total_cme_points = len(cme_indices)

    print("🛰️ CME Prediction Summary")
    print(f"-------------------------------------------")
    print(f"Predicted CME from index {cme_start_idx} to {cme_end_idx}")
    print(f"Predicted CME from {start_time} to {end_time}")
    print(f"Duration: {duration_hours:.2f} hours")
    print(f"Detected CME points: {total_cme_points}")
    print(f"Fraction of data labeled as CME: {total_cme_points / len(pred_labels):.4f}")
else:
    print("No CME predicted in this data.")


🛰️ CME Prediction Summary
-------------------------------------------
Predicted CME from index 49 to 863929
Predicted CME from 2025-10-17 00:00:11.316255 to 2025-10-18 00:00:00.078809
Duration: 24.00 hours
Detected CME points: 8516775
Fraction of data labeled as CME: 0.9858


/tmp/ipython-input-3567849051.py:16: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  start_time = datetime.utcfromtimestamp(cme_start_time)
/tmp/ipython-input-3567849051.py:17: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  end_time = datetime.utcfromtimestamp(cme_end_time)


In [ ]:
pred_labels = (predictions > 0.9).astype(int)

unique, counts = np.unique(pred_labels, return_counts=True)
print(dict(zip(unique, counts)))


NameError: name 'predictions' is not defined